In [1]:
! pip install langchain langchain_groq langchain_community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

groq_api_key = userdata.get('GROQ_API_KEY')


In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

In [4]:
llm.invoke("hi").content

'Hello! 👋\n\nHow can I help you today? 😊\n'

In [5]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

In [6]:
class State(TypedDict):
  messages: Annotated[List, add_messages]

In [8]:
graph = StateGraph(State)

def chatbot(state: State):
  return {"messages": [llm.invoke(state['messages'])]}

graph.add_node("chatbot", chatbot)
graph.set_entry_point("chatbot")
graph.set_finish_point("chatbot")

app = graph.compile()

In [15]:
def stream_app_updates(user_input: str):
  initial_state = {"messages":[("user", user_input)]}

  for event in app.stream(initial_state):
    for value in event.values():
      print("Assistant:", value['messages'][-1].content)

In [16]:
while True:
  try:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q", "exit"]:
      print("Good Bye!")
      break

    stream_app_updates(user_input)

  except Exception as e:
    print("Exceptions occured during processing:", e)
    stream_app_updates(user_input)
    break

User: hi
Assistant: Hi there! 👋

What can I do for you today?

User: how are you?
Assistant: As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have! Is there anything specific I can help you with today?

User: Who built Taj Mahal
Assistant: The Taj Mahal was built by **Mughal emperor Shah Jahan** in memory of his third wife, Mumtaz Mahal. 

User: q
Good Bye!
